In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import random
import torch
from typing import List, Dict, Optional
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
sys.path.append('..') 

from src.models.autoencoder import AutoEncoder
from src.models.autoencoder_trainer import *
from src.data.data_utils import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_dir = '../NETFLIX_DATA/partitions/train'
val_dir = '../NETFLIX_DATA/partitions/validation'
checkpoint_dir = '../checkpoints'

In [4]:
# retreive training data info
train_partition_files = get_data(data_dir)
print(f"Number of training partitions: {len(train_partition_files)}")
val_partition_files = get_data(val_dir)
print(f"Number of validation partitions: {len(val_partition_files)}")

seed = 42
random.seed(seed)

# testing
sample_train_partitions = random.sample(train_partition_files, 3)

sample_val_partitions = []
for partition in sample_train_partitions:
  val_partition = partition.copy()
  val_partition['path'] = partition['path'].replace('train', 'validation')
  sample_val_partitions.append(val_partition)


print(f"Train EX: {sample_train_partitions[0]}")
print(f"Val EX: {sample_val_partitions[0]}")

Number of training partitions: 34
Number of validation partitions: 34
Train EX: {'path': '../NETFLIX_DATA/partitions/train/part_1_7.parquet', 'part': 1, 'group': 7}
Val EX: {'path': '../NETFLIX_DATA/partitions/validation/part_1_7.parquet', 'part': 1, 'group': 7}


In [5]:
# build user_map and movie_map
user_map, movie_map = map_id(sample_train_partitions)

Mapping IDs: 100%|██████████| 3/3 [00:00<00:00,  6.63it/s]


Map successful for 406244 users, 1653 movies


In [6]:
# 7 each as a test + for tuning
train_partitions = sample_train_partitions
val_partitions = sample_val_partitions

In [7]:
params = {
    "num_epochs": 1,  #50
    "batch_size": 256,
    "learning_rate": 0.01, #0.001
    "hidden_dims": [256, 128], # 512 256 128
    "dropout": 0.5,
    "l2_reg": 0.001,
    "checkpoint_interval": 1, #5 or 10
    "eval_interval": 5,
}

In [8]:
# train model
model = train_autoencoder(
    train_partitions=sample_train_partitions,
    user_map=user_map,
    movie_map=movie_map,
    validation_partitions=sample_val_partitions,
    checkpoint_dir=checkpoint_dir,
    **params
)

Training on device: cuda


Calculating global mean: 100%|██████████| 3/3 [00:00<00:00, 20.51it/s]

Global mean rating: 3.582



Epoch 1/1: 100%|██████████| 1406/1406 [04:13<00:00,  5.54it/s, loss=1.4614]


Epoch 1 - Average Loss: 1.7536
Validation Loss: 3.0794
Saved checkpoint at ../checkpoints/checkpoint_epoch_1.pth.gz
Saved final model at ../checkpoints/final_model.pth.gz


In [9]:
validation_data = model.load_user_data(sample_val_partitions, user_map)

evaluate_model(
    model,
    validation_data=validation_data,
    movie_map=movie_map,
    device=device
)

Loading user data: 100%|██████████| 3/3 [00:00<00:00, 63.45it/s]


{'loss': 3.079413402570437,
 'rmse': 1.0926656718315744,
 'mae': 0.8589280956699702}